# Import necessary libraries

In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import os
import cv2
import matplotlib.pyplot as plt


## Define the custom Dataset class

In [2]:
# Define the custom Dataset class
class ImageDataset(Dataset):
    def __init__(self, directory, labels, transform=None):
        self.directory = directory  # Directory containing the images
        self.labels = labels  # List of labels (classes)
        self.transform = transform  # Transformations to be applied to images
        self.data = []  # List to hold image data and labels
        self.load_dataset()  # Load the dataset when initializing

    # Method to load images and their labels into the dataset
    def load_dataset(self):
        for idx, label in enumerate(self.labels):
            path = os.path.join(self.directory, label)
            files = os.listdir(path)
            for file in files:
                img_path = os.path.join(path, file)
                img = cv2.resize(cv2.imread(img_path), (50, 50))  # Resize images to 50x50 pixels
                self.data.append((img, idx))  # Append image and label index

    # Method to return the length of the dataset
    def __len__(self):
        return len(self.data)

    # Method to get an item (image and label) from the dataset by index
    def __getitem__(self, idx):
        img, label = self.data[idx]
        if self.transform:
            img = self.transform(img)  # Apply transformations if any
        return img, label


 ## Define transformations for data augmentation and normalization

In [3]:
#  Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert images to PIL format
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(10),  # Randomly rotate images by ±10 degrees
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize images
])


## Load dataset and create DataLoader instances

In [4]:
# Load dataset and create DataLoader instances
data_dir = r"C:\Users\rasha\Downloads\augmented_arabic_sign_language_dataset_small"
uniq_labels = sorted(os.listdir(data_dir))  # List of unique labels sorted alphabetically

# Create the dataset
dataset = ImageDataset(data_dir, uniq_labels, transform)

# Split the dataset into training, validation, and test sets (80/10/10)
train_size = int(0.8 * len(dataset))
validation_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - validation_size
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size, test_size])

# Create data loaders for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


##  Define the Convolutional Neural Network (CNN) model

In [5]:
# Define the Convolutional Neural Network (CNN) model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # First convolutional layer
            nn.ReLU(),  # ReLU activation
            nn.BatchNorm2d(16),  # Batch normalization
            nn.Conv2d(16, 16, kernel_size=3, padding=1),  # Second convolutional layer
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max pooling
            nn.Dropout(0.25)  # Dropout to prevent overfitting
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # Third convolutional layer
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),  # Fourth convolutional layer
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Fifth convolutional layer
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Sixth convolutional layer
            nn.ReLU(),
            nn.Dropout(0.25)
        )
        self.flatten = nn.Flatten()  # Flatten the feature maps for fully connected layers
        self.fc1 = nn.Linear(64 * 12 * 12, 128)  # First fully connected layer
        self.fc2 = nn.Linear(128, len(uniq_labels))  # Output layer

    # Define the forward pass of the network
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Set the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNet().to(device)


## Define loss function and optimizer

In [6]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Define the training function

In [7]:
# Define the training function
def train_model(model, train_loader, validation_loader, num_epochs=20):
    # Lists to store training and validation losses and accuracies
    train_losses, train_accs, val_losses, val_accs = [], [], [], []

    # Loop over the number of epochs
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        total_loss, total_correct, total_samples = 0, 0, 0  # Initialize counters

        # Loop over batches of training data
        for images, labels in train_loader:
            images = images.to(device)  # Move images to the device (GPU/CPU)
            labels = labels.to(device)  # Move labels to the device
            outputs = model(images)  # Forward pass: compute model output
            loss = criterion(outputs, labels)  # Compute loss
            optimizer.zero_grad()  # Zero the gradients
            loss.backward()  # Backward pass: compute gradients
            optimizer.step()  # Update model parameters
            total_loss += loss.item()  # Accumulate loss
            _, predicted = torch.max(outputs, 1)  # Get predicted labels
            total_correct += (predicted == labels).sum().item()  # Count correct predictions
            total_samples += labels.size(0)  # Count total samples

        # Calculate average training loss and accuracy for the epoch
        train_losses.append(total_loss / len(train_loader))
        train_accs.append(total_correct / total_samples)

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_loss, val_correct, val_samples = 0, 0, 0  # Initialize counters
        with torch.no_grad():  # Disable gradient calculation for validation
            for images, labels in validation_loader:
                images = images.to(device)  # Move images to the device
                labels = labels.to(device)  # Move labels to the device
                outputs = model(images)  # Forward pass: compute model output
                loss = criterion(outputs, labels)  # Compute loss
                val_loss += loss.item()  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get predicted labels
                val_correct += (predicted == labels).sum().item()  # Count correct predictions
                val_samples += labels.size(0)  # Count total samples

        # Calculate average validation loss and accuracy for the epoch
        val_losses.append(val_loss / len(validation_loader))
        val_accs.append(val_correct / val_samples)

        # Print epoch statistics
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, '
              f'Train Acc: {train_accs[-1]*100:.2f}%, Val Loss: {val_losses[-1]:.4f}, '
              f'Val Acc: {val_accs[-1]*100:.2f}%')

    return train_losses, train_accs, val_losses, val_accs


## Train the model

In [8]:
# Train the model
num_epochs = 30
train_losses, train_accs, val_losses, val_accs = train_model(model, train_loader, validation_loader, num_epochs)


Epoch 1/30, Train Loss: 3.1592, Train Acc: 10.34%, Val Loss: 2.8196, Val Acc: 17.94%
Epoch 2/30, Train Loss: 2.4253, Train Acc: 27.25%, Val Loss: 2.0125, Val Acc: 37.26%
Epoch 3/30, Train Loss: 2.0255, Train Acc: 37.70%, Val Loss: 1.7136, Val Acc: 45.74%
Epoch 4/30, Train Loss: 1.8121, Train Acc: 43.96%, Val Loss: 1.5855, Val Acc: 49.13%
Epoch 5/30, Train Loss: 1.6763, Train Acc: 47.90%, Val Loss: 1.4816, Val Acc: 54.45%
Epoch 6/30, Train Loss: 1.5676, Train Acc: 50.84%, Val Loss: 1.4263, Val Acc: 56.00%
Epoch 7/30, Train Loss: 1.4657, Train Acc: 53.47%, Val Loss: 1.3466, Val Acc: 56.90%
Epoch 8/30, Train Loss: 1.4172, Train Acc: 55.50%, Val Loss: 1.2655, Val Acc: 60.29%
Epoch 9/30, Train Loss: 1.3618, Train Acc: 57.15%, Val Loss: 1.2287, Val Acc: 61.94%


KeyboardInterrupt: 

 ## Plot the training and validation statistics

In [ ]:
# Plot the training and validation statistics
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accs, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), val_accs, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


## Plot the training and validation statistics

In [ ]:
# Save the trained model's state
torch.save(model.state_dict(), 'model_best_arabic_after_augmantation.ckpt')
